In [ ]:
import pyopencl as cl
from pyopencl.reduction import ReductionKernel
from pyopencl import array
import numpy as np
import time
import datetime
import os.path


In [ ]:
platform = cl.get_platforms()[0]
device1 = platform.get_devices()[1]
device2 = platform.get_devices()[2]
context = cl.Context([device1, device2])

In [ ]:
tic = time.clock()

scale = 4
corrosion_depth = 5
dt = 0.01

# Define time paramters
T_h = 0.5     # heating time
T = 0.5    # total simulation time
theta = 1
tol = 1E-6

# Define physical parameters, length scale is mm
T_init = 0
P = 10e9
omega = 2.
vert_scale = np.array([-15, -15, 0, 1./scale, 1./scale, 1./scale]).astype(np.float32)

# Define material parameters
rho_0 = 0.0076; rho_1 = 0.003;
C_0 = 4.9e8; C_1 = 5.5e8;
k_0 = 4.3e7; k_1 = 4e6;
mat_coefs = np.array([rho_0*C_0, k_0, rho_1*C_1, k_1]).astype(np.float32)

# Other FE arrays
C = np.array([30*scale, 30*scale, 10*scale]).astype(np.uint32)
nCells = C[0]*C[1]*C[2]*6
nVertices = (C[0]+1)*(C[1]+1)*(C[2]+1)
DoFMapLocal = np.array([0, 1, 3, 7,
                       0, 1, 5, 7,
                       0, 4, 5, 7,
                       0, 2, 3, 7,
                       0, 4, 6, 7,
                       0, 2, 6, 7]).astype(np.uint32)
DoFMapLocal2 = np.array([1, 33,
                   1, 65,
                   64, 65,
                   32, 33,
                   64, 96,
                   32, 96]).astype(np.uint32)
zMapLocal = np.array([0, 0,
                   0, vert_scale[5],
                   vert_scale[5], vert_scale[5],
                   0, 0,
                   vert_scale[5], vert_scale[5],
                   0, vert_scale[5]]).astype(np.float32)
with np.load('Scale_{}/meshData.npz'.format(scale)) as data:
    Mdata = data['Mdata']
    Kdata = data['Kdata']

M_FG = np.zeros(4, cl.array.vec.float4)
K_FG = np.zeros(4, cl.array.vec.float4)
for i in range(4):
    M_FG[i] = (Mdata[4*i + 0], Mdata[4*i + 1], Mdata[4*i + 2], Mdata[4*i + 3])
    K_FG[i] = (Kdata[4*i + 0], Kdata[4*i + 1], Kdata[4*i + 2], Kdata[4*i + 3])

Ndt = np.zeros(nVertices).astype(np.float32)
Ndt[0:(C[0]+1)*(C[1]+1)-1] = 1
# Ndt = N_host
# Ndt[(C[0]+1)*(C[1]+1):nVertices] = 0
# Ndt = N*dt
# Ndt = (Ndt>0).astype(np.float32)

x = np.zeros(nVertices).astype(np.float32)
u_0 = T_init*np.ones_like(x)
Ax_split = np.zeros(nVertices*4).astype(np.float32)
P_split = np.zeros(nVertices*4).astype(np.float32)
corr_bounds = np.array([corrosion_depth]).astype(np.float32)
theta = np.float32(theta)
dt = np.float32(dt)
delta = np.array([1,]).astype(np.float32)
delta_new = np.empty_like(delta)
nC_with_padding = (C[0]+1)*(C[1]+1)*C[2]*6

toc = time.clock()
print "load data                {}".format(toc - tic)

In [ ]:
delta1_dev1 = np.array([1,]).astype(np.float32)
delta2_dev2 = np.array([1,]).astype(np.float32)
delta1_new_dev1 = np.ones_like(delta1_dev1)
delta2_new_dev2 = np.ones_like(delta2_dev2)
gamma1_dev1 = np.ones_like(delta1_dev1)
gamma2_dev2 = np.ones_like(delta2_dev2)
neg_gamma1_dev1 = np.ones_like(delta1_dev1)
neg_gamma2_dev2 = np.ones_like(delta2_dev2)

In [ ]:
compiler_options = '-cl-mad-enable -cl-fast-relaxed-math'
# compiler_options = ''
Jacobi_A_prg = cl.Program(context, """
    __kernel void J_A(__constant float4 *M_FG, __constant float4 *K_FG,
    __constant uint *DoFMapLocal2,
    __constant float *zMapLocal,
    __constant uint *C,
    __constant float *vertex_loc_scale,
    __constant float *corr_bounds,
    __constant float *material_coefs,
    float theta, float dt,
    __local float *P_split_local,
    __global float *P_split)
     {
        uint gid = get_global_id(0);
        uint wid = get_group_id(0);
        uint lid = get_local_id(0);
        uint numgroups = get_num_groups(0);
        
        uint global_cell6id = (gid)/6 - wid;
        uint local_cell6id = lid/6;
        uint global_read_row = lid/32;
        uint private_cell6id = lid % 6;
        
        uint Cx = C[0];
        uint Cy = C[1];
        uint Cz = C[2];
        uint CxV = Cx+1;
        uint CyV = Cy+1;
        uint CzV = Cz+1;
        
        uint wg_base_id = wid*30;
        uint wgZ = wg_base_id/(CxV*CyV);
        uint wgY = (wg_base_id - wgZ*(CxV*CyV))/CxV;
        uint wgX = wg_base_id - wgZ*CxV*CyV - wgY*CxV;
        uint wg2g_offset = wgX + CxV*wgY + CxV*CyV*wgZ;

        uint global_read_ind;
        bool global_read = true;
        switch (global_read_row){
            case 0:
                global_read_ind = wg2g_offset + lid;
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            case 1:
                global_read_ind = wg2g_offset + (lid-32) + CxV;
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            case 2:
                global_read_ind = wg2g_offset + (lid-64) + CxV*CyV;
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            case 3:
                global_read_ind = wg2g_offset + (lid-96) + CxV*(CyV+1);
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            default:
                global_read = false;
        }
        
        uint nCells = CxV*CyV*Cz*6;
        uint i, j, k, l;
        i = local_cell6id;
        j = local_cell6id + DoFMapLocal2[2*private_cell6id];
        k = local_cell6id + DoFMapLocal2[2*private_cell6id + 1];
        l = local_cell6id + 97;

        uint c6Z = global_cell6id/(CxV*CyV);
        uint c6Y = (global_cell6id - c6Z*(CxV*CyV))/CxV;
        uint c6X = global_cell6id - c6Z*CxV*CyV - c6Y*CxV;
        
        float vX = (c6X * vertex_loc_scale[3]) + vertex_loc_scale[0];
        float vY = (c6Y * vertex_loc_scale[4]) + vertex_loc_scale[1];
        float vZ = (c6Z * vertex_loc_scale[5]) + vertex_loc_scale[2];
        
        float M_coef = 0;
        float K_coef = 0;
        if(vZ>corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + vertex_loc_scale[5] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id + 1] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        
        M_coef = M_coef/4.f;
        K_coef = K_coef/4.f;
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        bool cell_in_bounds = (gid - 6*wid < nCells);
        float P_split_local_contribution;
        if(cell_in_bounds && (c6X < CxV-1) && (c6Y < CyV-1)){
            P_split_local_contribution = M_coef*M_FG[0].s0 + theta*dt*K_coef*K_FG[0].s0;
            P_split_local[i*12 + private_cell6id] = P_split_local_contribution;
            P_split_local_contribution = M_coef*M_FG[1].s1 + theta*dt*K_coef*K_FG[1].s1;
            P_split_local[j*12 + private_cell6id + 6] = P_split_local_contribution;
            P_split_local_contribution = M_coef*M_FG[2].s2 + theta*dt*K_coef*K_FG[2].s2;
            P_split_local[k*12 + private_cell6id] = P_split_local_contribution;
            P_split_local_contribution = M_coef*M_FG[3].s3 + theta*dt*K_coef*K_FG[3].s3;
            P_split_local[l*12 + private_cell6id + 6] = P_split_local_contribution;}
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        uint global_write_ind = global_read_ind*4 + global_read_row;
        bool vertex_in_bounds = (global_write_ind < CxV*CyV*CzV*4);
        float P_split_contribution = 0;
        if(global_read && vertex_in_bounds &&
        ((lid % 32 < 31) || (wid == numgroups-1)) && ((lid % 32 > 0) || (wid == 0))){
        for(uint vertex_neigbor = 0; vertex_neigbor < 12; vertex_neigbor++){
            P_split_contribution += P_split_local[lid*12 + vertex_neigbor];
        }
            P_split[global_write_ind] = P_split_contribution;
        }
    }
    """).build(options=compiler_options)

FGEbEMVP_A_prg = cl.Program(context, """
    __kernel void FGEbEMVP_A(__constant float4 *M_FG, __constant float4 *K_FG, 
    __global float *x,
    __constant uint *DoFMapLocal2,
    __constant float *zMapLocal,
    __constant uint *C,
    __constant float *vertex_loc_scale,
    __constant float *corr_bounds,
    __constant float *material_coefs,
    float theta, float dt,
    __local float *x_local,
    __local float *Ax_split_local,
    __global float *Ax_split)
    {
        uint gid = get_global_id(0);
        uint wid = get_group_id(0);
        uint lid = get_local_id(0);
        uint numgroups = get_num_groups(0);
        
        uint global_cell6id = (gid)/6 - wid;
        uint local_cell6id = lid/6;
        uint global_read_row = lid/32;
        uint private_cell6id = lid % 6;
        
        uint Cx = C[0];
        uint Cy = C[1];
        uint Cz = C[2];
        uint CxV = Cx+1;
        uint CyV = Cy+1;
        uint CzV = Cz+1;
        
        uint wg_base_id = wid*30;
        uint wgZ = wg_base_id/(CxV*CyV);
        uint wgY = (wg_base_id - wgZ*(CxV*CyV))/CxV;
        uint wgX = wg_base_id - wgZ*CxV*CyV - wgY*CxV;
        uint wg2g_offset = wgX + CxV*wgY + CxV*CyV*wgZ;

        uint global_read_ind;
        bool global_read = true;
        switch (global_read_row){
            case 0:
                global_read_ind = wg2g_offset + lid;
                break;
            case 1:
                global_read_ind = wg2g_offset + (lid-32) + CxV;
                break;
            case 2:
                global_read_ind = wg2g_offset + (lid-64) + CxV*CyV;
                break;
            case 3:
                global_read_ind = wg2g_offset + (lid-96) + CxV*(CyV+1);
                break;
            default:
                global_read = false;
        }
        
        if(global_read){
            x_local[lid] = x[global_read_ind];
        }
        
         switch (global_read_row){
            case 0:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
            case 1:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
            case 2:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
            case 3:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
        }
        
        uint nCells = CxV*CyV*Cz*6;
        uint i, j, k, l;
        i = local_cell6id;
        j = local_cell6id + DoFMapLocal2[2*private_cell6id];
        k = local_cell6id + DoFMapLocal2[2*private_cell6id + 1];
        l = local_cell6id + 97;

        uint c6Z = global_cell6id/(CxV*CyV);
        uint c6Y = (global_cell6id - c6Z*(CxV*CyV))/CxV;
        uint c6X = global_cell6id - c6Z*CxV*CyV - c6Y*CxV;
        
        float vX = (c6X * vertex_loc_scale[3]) + vertex_loc_scale[0];
        float vY = (c6Y * vertex_loc_scale[4]) + vertex_loc_scale[1];
        float vZ = (c6Z * vertex_loc_scale[5]) + vertex_loc_scale[2];
        
        float M_coef = 0;
        float K_coef = 0;
        if(vZ>corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + vertex_loc_scale[5] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id + 1] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        
        M_coef = M_coef/4.f;
        K_coef = K_coef/4.f;
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        float Mx, Kx;
        
        bool cell_in_bounds = (gid - 6*wid < nCells);
        if(cell_in_bounds && (c6X < CxV-1) && (c6Y < CyV-1)){
            float4 x_cell = (float4)(x_local[i], x_local[j], x_local[k], x_local[l]);
            Mx = dot((M_FG[0]), x_cell); Kx = dot(K_FG[0], x_cell);
            float Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[i*12 + private_cell6id] = Ax_split_local_contribution;
            Mx = dot((M_FG[1]), x_cell); Kx = dot(K_FG[1], x_cell);
            Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[j*12 + private_cell6id + 6] = Ax_split_local_contribution;
            Mx = dot((M_FG[2]), x_cell); Kx = dot(K_FG[2], x_cell);
            Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[k*12 + private_cell6id] = Ax_split_local_contribution;
            Mx = dot((M_FG[3]), x_cell); Kx = dot(K_FG[3], x_cell);
            Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[l*12 + private_cell6id + 6] = Ax_split_local_contribution;}
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        uint global_write_ind = global_read_ind*4 + global_read_row;
        bool vertex_in_bounds = (global_write_ind < CxV*CyV*CzV*4);
        float Ax_split_contribution = 0;
        if(global_read && vertex_in_bounds &&
        ((lid % 32 < 31) || (wid == numgroups-1)) && ((lid % 32 > 0) || (wid == 0))){
        for(uint vertex_neigbor = 0; vertex_neigbor < 12; vertex_neigbor++){
            Ax_split_contribution += Ax_split_local[lid*12 + vertex_neigbor];
        }
            Ax_split[global_write_ind] = Ax_split_contribution;
        }
        
    }
    """).build(options=compiler_options) 

FGEbEMVP_B_prg = cl.Program(context, """
    __kernel void FGEbEMVP_B(__global float *Ax_split,
    __global float *Ax)
    {
        uint gid = get_global_id(0);
        uint vertex_offset = gid * 4;
        float Ax_contribution = 0;
        
        for(uint i = 0; i < 4; i++){
            Ax_contribution += Ax_split[vertex_offset + i];
        }

        Ax[gid] = Ax_contribution;     
    }
    """).build(options=compiler_options)

FGEbEMVP_C_prg = cl.Program(context, """
    __kernel void FGEbEMVP_C(__global float *Ax_split,
    __global float *b,
    float c,
    __global float *cAx_plus_b)
    {
        uint gid = get_global_id(0);
        uint vertex_offset = gid * 4;
        float Ax_contribution = b[gid];
        
        for(uint i = 0; i < 4; i++){
            Ax_contribution += c*Ax_split[vertex_offset + i];
        }

        cAx_plus_b[gid] = Ax_contribution;     
    }
    """).build(options=compiler_options)

## Added offset
VVP_A_prg = cl.Program(context, """
    __kernel void VVP_A(__global float *x,
    __global float *y,
    uint offset,
    uint N,
    __local float *VVP_loc,
    __global float *r)
    {
      uint gid = get_global_id(0) + offset;
      uint wid = get_group_id(0);
      uint lid = get_local_id(0);
      uint gs = get_local_size(0);
      if(gid < N + offset) VVP_loc[lid] = (x[gid] * y[gid]);
      else VVP_loc[lid] = 0;
      barrier(CLK_LOCAL_MEM_FENCE);
      for(uint s = gs/2; s > 0; s >>= 1) {
        if(lid < s) {
          VVP_loc[lid] += VVP_loc[lid+s];
        }
        barrier(CLK_LOCAL_MEM_FENCE);
      }
      if(lid == 0) r[wid] = VVP_loc[lid];
    }
    """).build(options=compiler_options)

VVP_C_prg = cl.Program(context, """
    __kernel void VVP_C(__global float *r,
    uint N,
    __local float *VVP_loc,
    __global float *gamma,
    __global float *neg_gamma)
    {
      uint gid = get_global_id(0);
      uint wid = get_group_id(0);
      uint lid = get_local_id(0);
      uint gs = get_local_size(0);
      if(gid < N) VVP_loc[lid] = r[gid];
      else VVP_loc[lid] = 0;
      barrier(CLK_LOCAL_MEM_FENCE);
      for(uint s = gs/2; s > 0; s >>= 1) {
        if(lid < s) {
          VVP_loc[lid] += VVP_loc[lid+s];
        }
        barrier(CLK_LOCAL_MEM_FENCE);
      }
      if(lid == 0){
          float gamma_temp = VVP_loc[lid];
          gamma[0] = VVP_loc[lid];
          neg_gamma[0] = -gamma_temp;
      }
    }
    """).build(options=compiler_options)

VVP_reduce_prg = cl.Program(context, """
    __kernel void VVP_reduce(__global float *r1,
    uint N,
    __local float *VVP_loc,
    __global float *r2)
    {
      uint gid = get_global_id(0);
      uint wid = get_group_id(0);
      uint lid = get_local_id(0);
      uint gs = get_local_size(0);
      if(gid < N) VVP_loc[lid] = r1[gid];
      else VVP_loc[lid] = 0;
      barrier(CLK_LOCAL_MEM_FENCE);
      for(uint s = gs/2; s > 0; s >>= 1) {
        if(lid < s) {
          VVP_loc[lid] += VVP_loc[lid+s];
        }
        barrier(CLK_LOCAL_MEM_FENCE);
      }
      if(lid == 0) r2[wid] = VVP_loc[lid];
    }
    """).build(options=compiler_options)

VAVSP_prg = cl.Program(context, """
    __kernel void VAVSP(__global float *x,
    __global float *y,
    __constant float *num1,
    __constant float *num2,
    __constant float *denom1,
    __constant float *denom2,
    __global float *x_plus_ay)
    {
        int gid = get_global_id(0);
        float x_temp;
        float y_temp;
        float c_temp;
        x_temp = x[gid];
        y_temp = y[gid];
        
        c_temp = x_temp + (num1[0] + num2[0])/(denom1[0] + denom2[0])*y_temp;
        x_plus_ay[gid] = c_temp;     
    }
    """).build(options=compiler_options)

DIMVP_prg = cl.Program(context, """
    __kernel void DIMVP(__global float *P,
    __global float *x,
    __global float *Pinvx)
    {
        int gid = get_global_id(0);
        Pinvx[gid] = x[gid]/P[gid];     
    }
    """).build(options=compiler_options)

u0_update_prg = cl.Program(context, """
    __kernel void u0_update(__global float *u0,
    __global float *u_new)
    {
        int gid = get_global_id(0);
        u0[gid] = u_new[gid];
        u_new[gid] = 2*u_new[gid] - u0[gid];
    }
    """).build(options=compiler_options)


In [ ]:
queue1  = cl.CommandQueue(context, device = device1)
queue2  = cl.CommandQueue(context, device = device2)
mf = cl.mem_flags
map_flags = cl.map_flags
MVP_wg_size = np.uint32(6*31)
MVP_global_size = np.uint32(np.ceil((nC_with_padding/(MVP_wg_size-6.))) * MVP_wg_size)
max_wg_size = context.get_info(cl.context_info.DEVICES)[0].max_work_group_size

# Fraction to be computed on device1
m_frac = .5
m_slices = np.uint32(np.ceil(C[2]*m_frac))
slice_size = np.uint32((C[0]+1)*(C[1]+1))
m = np.uint32((m_slices+1)*slice_size)

# Vertices need for devices, including overlap
m1 = np.uint32((m_slices + 2)*slice_size)
m2 = np.uint32((C[2] - m_slices + 1)*slice_size)
m2_inner = np.uint32(m2 - slice_size)
m1_inner = np.uint32(m1 - slice_size)

# Split cell dimensions
C1 = np.array([C[0], C[1], m_slices + 1]).astype(np.uint32)
nC1 = (C1[0]+1)*(C1[1]+1)*C1[2]*6
MVP_global_size1 = np.uint32(np.ceil((nC1 /(MVP_wg_size-6.)) ) * MVP_wg_size)
C2 = np.array([C[0], C[1], C[2] - m_slices]).astype(np.uint32)
nC2 = (C2[0]+1)*(C2[1]+1)*C2[2]*6
MVP_global_size2 = np.uint32(np.ceil((nC2 /(MVP_wg_size-6.)) ) * MVP_wg_size)

# Split spatial domain
vertex_loc_scale1 = vert_scale
vertex_loc_scale2 = np.array([vert_scale[0], vert_scale[1], 
                              vert_scale[2] + vert_scale[5]*m_slices,
                              vert_scale[3], vert_scale[4],
                              vert_scale[5]]).astype(np.float32)


HOST_TO_DEVICE_COPY = (mf.READ_ONLY  | mf.HOST_WRITE_ONLY | mf.COPY_HOST_PTR)
HOST_TO_DEVICE_USE  = (mf.READ_ONLY  | mf.HOST_WRITE_ONLY | mf.USE_HOST_PTR)
HOST_READ_WRITE     = (mf.READ_WRITE | mf.COPY_HOST_PTR)
PINNED              = (mf.READ_WRITE | mf.USE_HOST_PTR)
DEVICE_READ_WRITE   = (mf.READ_WRITE | mf.HOST_NO_ACCESS)

M_FG_buf         = cl.Buffer(context, HOST_TO_DEVICE_USE, hostbuf = M_FG)
K_FG_buf         = cl.Buffer(context, HOST_TO_DEVICE_USE, hostbuf = K_FG)
DoFMapLocal2_buf = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = DoFMapLocal2)
zMapLocal_buf    = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = zMapLocal)
corr_bounds_buf  = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = corr_bounds)
mat_coefs_buf    = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = mat_coefs)

C_buf1           = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = C1)
vert_scale_buf1  = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = vertex_loc_scale1)
C_buf2           = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = C2)
vert_scale_buf2  = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = vertex_loc_scale2)

Ax_split_buf1    = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros_like(Ax_split))
P_split_buf1     = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros_like(P_split))
P_buf1           = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.ones_like(x))
Ax_split_buf2    = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros_like(Ax_split))
P_split_buf2     = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros_like(P_split))
P_buf2           = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.ones_like(x))

Ndt_buf1          = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = Ndt[0:m1])
x_init_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = u_0[0:m1])
b_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m1,)).astype(np.float32))
Ndt_buf2          = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = Ndt[nVertices-m2:nVertices])
x_init_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = u_0[0:m1])
b_buf2            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m2,)).astype(np.float32))

u0_buf1           = cl.Buffer(context, HOST_READ_WRITE, hostbuf = u_0[0:m1])
u0_buf2           = cl.Buffer(context, HOST_READ_WRITE, hostbuf = u_0[nVertices-m2:nVertices])
x_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = x[0:m1])
x_buf2            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = x[nVertices-m2:nVertices])
d_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m1,)).astype(np.float32))
d_buf2            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m2,)).astype(np.float32))

d_boundary_buf1 = cl.Buffer(context, PINNED, hostbuf = np.zeros((slice_size,)).astype(np.float32))
d_boundary_buf2 = cl.Buffer(context, PINNED, hostbuf = np.zeros((slice_size,)).astype(np.float32))
u0_x_boundary_buf1 = cl.Buffer(context, PINNED, hostbuf = np.zeros((2*slice_size,)).astype(np.float32))
u0_x_boundary_buf2 = cl.Buffer(context, PINNED, hostbuf = np.zeros((2*slice_size,)).astype(np.float32))

r_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m1,)).astype(np.float32))
q_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m1,)).astype(np.float32))
s_buf1            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m1,)).astype(np.float32))
r_buf2            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m2,)).astype(np.float32))
q_buf2            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m2,)).astype(np.float32))
s_buf2            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((m2,)).astype(np.float32))

delta1_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = delta1_dev1)
delta2_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = delta2_dev2)
delta1_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = delta1_dev1)
delta2_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = delta2_dev2)
delta1_new_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = delta1_new_dev1)
delta2_new_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = delta2_new_dev2)
delta1_new_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = delta1_new_dev1)
delta2_new_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = delta2_new_dev2)
gamma1_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = gamma1_dev1)
gamma2_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = gamma2_dev2)
gamma1_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = gamma1_dev1)
gamma2_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = gamma2_dev2)
neg_gamma1_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = neg_gamma1_dev1)
neg_gamma2_dev1_buf  = cl.Buffer(context, PINNED, hostbuf = neg_gamma2_dev2)
neg_gamma1_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = neg_gamma1_dev1)
neg_gamma2_dev2_buf  = cl.Buffer(context, PINNED, hostbuf = neg_gamma2_dev2)

x_local_buf1        = cl.LocalMemory(4 * 32*4)
Ax_split_local_buf1 = cl.LocalMemory(4 * 32*4*12)
VVP_loc_buf1        = cl.LocalMemory(4 * max_wg_size)
x_local_buf2        = cl.LocalMemory(4 * 32*4)
Ax_split_local_buf2 = cl.LocalMemory(4 * 32*4*12)
VVP_loc_buf2        = cl.LocalMemory(4 * max_wg_size)

knl_PA_1 = Jacobi_A_prg.J_A
knl_PA_1.set_args(M_FG_buf, K_FG_buf, DoFMapLocal2_buf, zMapLocal_buf, C_buf1,
                vert_scale_buf1, corr_bounds_buf, mat_coefs_buf, (np.float32(1)*theta), 
                dt, Ax_split_local_buf1, P_split_buf1)
knl_PB_1 = FGEbEMVP_B_prg.FGEbEMVP_B
knl_PB_1.set_args(P_split_buf1, P_buf1)
knl_PA_2 = Jacobi_A_prg.J_A
knl_PA_2.set_args(M_FG_buf, K_FG_buf, DoFMapLocal2_buf, zMapLocal_buf, C_buf2,
                vert_scale_buf2, corr_bounds_buf, mat_coefs_buf, (np.float32(1)*theta), 
                dt, Ax_split_local_buf2, P_split_buf2)
knl_PB_2 = FGEbEMVP_B_prg.FGEbEMVP_B
knl_PB_2.set_args(P_split_buf2, P_buf2)

knl_RHS_A_1 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_RHS_A_1.set_args(M_FG_buf, K_FG_buf, u0_buf1, DoFMapLocal2_buf, zMapLocal_buf, C_buf1,
                vert_scale_buf1, corr_bounds_buf, mat_coefs_buf, (np.float32(-(1-theta))), 
                dt, x_local_buf1, Ax_split_local_buf1, Ax_split_buf1)
knl_RHS_B_1 = FGEbEMVP_C_prg.FGEbEMVP_C
knl_RHS_B_1.set_args(Ax_split_buf1, Ndt_buf1, np.float32(1), b_buf1)
knl_RHS_A_2 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_RHS_A_2.set_args(M_FG_buf, K_FG_buf, u0_buf2, DoFMapLocal2_buf, zMapLocal_buf, C_buf2,
                vert_scale_buf2, corr_bounds_buf, mat_coefs_buf, (np.float32(-(1-theta))), 
                dt, x_local_buf2, Ax_split_local_buf2, Ax_split_buf2)
knl_RHS_B_2 = FGEbEMVP_C_prg.FGEbEMVP_C
knl_RHS_B_2.set_args(Ax_split_buf2, Ndt_buf2, np.float32(1), b_buf2)

knl_1A_1 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_1A_1.set_args(M_FG_buf, K_FG_buf, x_buf1, DoFMapLocal2_buf, zMapLocal_buf, C_buf1,
                vert_scale_buf1, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf1, Ax_split_local_buf1, Ax_split_buf1)
knl_1B_1 = FGEbEMVP_C_prg.FGEbEMVP_C
knl_1B_1.set_args(Ax_split_buf1, b_buf1, np.float32(-1), r_buf1)
knl_1A_2 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_1A_2.set_args(M_FG_buf, K_FG_buf, x_buf2, DoFMapLocal2_buf, zMapLocal_buf, C_buf2,
                vert_scale_buf2, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf2, Ax_split_local_buf2, Ax_split_buf2)
knl_1B_2 = FGEbEMVP_C_prg.FGEbEMVP_C
knl_1B_2.set_args(Ax_split_buf2, b_buf2, np.float32(-1), r_buf2)

knl_2_1 = DIMVP_prg.DIMVP
knl_2_1.set_args(P_buf1, r_buf1, d_buf1)
knl_2_2 = DIMVP_prg.DIMVP
knl_2_2.set_args(P_buf2, r_buf2, d_buf2)

knl_4A_1 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_4A_1.set_args(M_FG_buf, K_FG_buf, d_buf1, DoFMapLocal2_buf, zMapLocal_buf, C_buf1,
                vert_scale_buf1, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf1, Ax_split_local_buf1, Ax_split_buf1)
knl_4B_1 = FGEbEMVP_B_prg.FGEbEMVP_B
knl_4B_1.set_args(Ax_split_buf1, q_buf1)
knl_4A_2 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_4A_2.set_args(M_FG_buf, K_FG_buf, d_buf2, DoFMapLocal2_buf, zMapLocal_buf, C_buf2,
                vert_scale_buf2, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf2, Ax_split_local_buf2, Ax_split_buf2)
knl_4B_2 = FGEbEMVP_B_prg.FGEbEMVP_B
knl_4B_2.set_args(Ax_split_buf2, q_buf2)

knl_6_1 = VAVSP_prg.VAVSP
knl_6_1.set_args(x_buf1, d_buf1, delta1_new_dev1_buf, delta2_new_dev1_buf,
                 gamma1_dev1_buf, gamma2_dev1_buf, x_buf1)
knl_6_2 = VAVSP_prg.VAVSP
knl_6_2.set_args(x_buf2, d_buf2, delta1_new_dev2_buf, delta2_new_dev2_buf,
                 gamma1_dev2_buf, gamma2_dev2_buf, x_buf2)

knl_7A_1 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_7A_1.set_args(M_FG_buf, K_FG_buf, x_buf1, DoFMapLocal2_buf, zMapLocal_buf, C_buf1,
                vert_scale_buf1, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf1, Ax_split_local_buf1, Ax_split_buf1)
knl_7B_1 = FGEbEMVP_C_prg.FGEbEMVP_C
knl_7B_1.set_args(Ax_split_buf1, b_buf1, np.float32(-1), r_buf1)
knl_7A_2 = FGEbEMVP_A_prg.FGEbEMVP_A
knl_7A_2.set_args(M_FG_buf, K_FG_buf, x_buf2, DoFMapLocal2_buf, zMapLocal_buf, C_buf2,
                vert_scale_buf2, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf2, Ax_split_local_buf2, Ax_split_buf2)
knl_7B_2 = FGEbEMVP_C_prg.FGEbEMVP_C
knl_7B_2.set_args(Ax_split_buf2, b_buf2, np.float32(-1), r_buf2)

knl_7_1 = VAVSP_prg.VAVSP
knl_7_1.set_args(r_buf1, q_buf1, delta1_new_dev1_buf, delta2_new_dev1_buf,
                 neg_gamma1_dev1_buf, neg_gamma2_dev1_buf, r_buf1)
knl_7_2 = VAVSP_prg.VAVSP
knl_7_2.set_args(r_buf2, q_buf2, delta1_new_dev2_buf, delta2_new_dev2_buf,
                 neg_gamma1_dev2_buf, neg_gamma2_dev2_buf, r_buf2)

knl_8_1 = DIMVP_prg.DIMVP
knl_8_1.set_args(P_buf1, r_buf1, s_buf1)
knl_8_2 = DIMVP_prg.DIMVP
knl_8_2.set_args(P_buf2, r_buf2, s_buf2)

knl_11_1 = VAVSP_prg.VAVSP
knl_11_1.set_args(s_buf1, d_buf1, delta1_new_dev1_buf, delta2_new_dev1_buf,
                 delta1_dev1_buf, delta2_dev1_buf, d_buf1)
knl_11_2 = VAVSP_prg.VAVSP
knl_11_2.set_args(s_buf2, d_buf2, delta1_new_dev2_buf, delta2_new_dev2_buf,
                 delta1_dev2_buf, delta2_dev2_buf, d_buf2)

knl_u0_1 = u0_update_prg.u0_update
knl_u0_1.set_args(u0_buf1, x_buf1)
knl_u0_2 = u0_update_prg.u0_update
knl_u0_2.set_args(u0_buf2, x_buf2)

if (np.min([m1,m2]) <= max_wg_size * max_wg_size):
    reduction_steps = 1
    r1_size1 = np.uint32(np.ceil(m1_inner/max_wg_size))
    global_red_size1 = r1_size1 * max_wg_size
    r1_size2 = np.uint32(np.ceil(m2_inner/max_wg_size))
    global_red_size2 = r1_size2 * max_wg_size
    
    r1_buf1 = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((r1_size1,)))
    r1_buf2 = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros((r1_size2,)))
    
    knl_3A_1 = VVP_A_prg.VVP_A
    knl_3A_1.set_args(r_buf1, d_buf1, np.uint32(0), m1_inner, VVP_loc_buf1, r1_buf1)
    knl_3B_1 = VVP_reduce_prg.VVP_reduce
    knl_3B_1.set_args(r1_buf1, r1_size1, VVP_loc_buf1, delta1_new_dev1_buf)
    knl_5A_1 = VVP_A_prg.VVP_A
    knl_5A_1.set_args(d_buf1, q_buf1, np.uint32(0), m1_inner, VVP_loc_buf1, r1_buf1)
    knl_5B_1 = VVP_C_prg.VVP_C
    knl_5B_1.set_args(r1_buf1, r1_size1, VVP_loc_buf1, 
                      gamma1_dev1_buf, neg_gamma1_dev1_buf)
    knl_9A_1 = VVP_A_prg.VVP_A
    knl_9A_1.set_args(r_buf1, s_buf1, np.uint32(0), m1_inner, VVP_loc_buf1, r1_buf1)
    knl_9B_1 = VVP_reduce_prg.VVP_reduce
    knl_9B_1.set_args(r1_buf1, r1_size1, VVP_loc_buf1, delta1_new_dev1_buf) 
    
    knl_3A_2 = VVP_A_prg.VVP_A
    knl_3A_2.set_args(r_buf2, d_buf2, slice_size, m2_inner, VVP_loc_buf2, r1_buf2)
    knl_3B_2 = VVP_reduce_prg.VVP_reduce
    knl_3B_2.set_args(r1_buf2, r1_size2, VVP_loc_buf2, delta2_new_dev2_buf)
    knl_5A_2 = VVP_A_prg.VVP_A
    knl_5A_2.set_args(d_buf2, q_buf2, slice_size, m2_inner, VVP_loc_buf2, r1_buf2)
    knl_5B_2 = VVP_C_prg.VVP_C
    knl_5B_2.set_args(r1_buf2, r1_size2, VVP_loc_buf2, 
                      gamma2_dev2_buf, neg_gamma2_dev2_buf)
    knl_9A_2 = VVP_A_prg.VVP_A
    knl_9A_2.set_args(r_buf2, s_buf2, slice_size, m2_inner, VVP_loc_buf2, r1_buf2)
    knl_9B_2 = VVP_reduce_prg.VVP_reduce
    knl_9B_2.set_args(r1_buf2, r1_size2, VVP_loc_buf2, delta2_new_dev2_buf)
    
elif(np.min([m1,m2]) <= max_wg_size * max_wg_size * max_wg_size):
    reduction_steps = 2
    r1_size1 = np.uint32(np.ceil(m1_inner/max_wg_size))
    global_red1_size1 = r1_size1 * max_wg_size
    r2_size1 = np.uint32(np.ceil(r1_size1/max_wg_size))
    global_red2_size1 = r2_size1 * max_wg_size
    
    r1_size2 = np.uint32(np.ceil(m2_inner/max_wg_size))
    global_red1_size2 = r1_size2 * max_wg_size
    r2_size2 = np.uint32(np.ceil(r1_size2/max_wg_size))
    global_red2_size2 = r2_size2 * max_wg_size
    
    r1_buf1 = cl.Buffer(context, DEVICE_READ_WRITE, r1_size1*4)
    r2_buf1 = cl.Buffer(context, DEVICE_READ_WRITE, r2_size1*4)
    r1_buf2 = cl.Buffer(context, DEVICE_READ_WRITE, r1_size2*4)
    r2_buf2 = cl.Buffer(context, DEVICE_READ_WRITE, r2_size2*4)
    
    knl_3A_1 = VVP_A_prg.VVP_A
    knl_3A_1.set_args(r_buf1, d_buf1, np.uint32(0), m1_inner, VVP_loc_buf1, r1_buf1)
    knl_3B_1 = VVP_reduce_prg.VVP_reduce
    knl_3B_1.set_args(r1_buf1, r1_size1, VVP_loc_buf1, r2_buf1)
    knl_3C_1 = VVP_reduce_prg.VVP_reduce
    knl_3C_1.set_args(r2_buf1, r2_size1, VVP_loc_buf1, delta1_new_dev1_buf)
    knl_5A_1 = VVP_A_prg.VVP_A
    knl_5A_1.set_args(d_buf1, q_buf1, np.uint32(0), m1_inner, VVP_loc_buf1, r1_buf1)
    knl_5B_1 = VVP_reduce_prg.VVP_reduce
    knl_5B_1.set_args(r1_buf1, r1_size1, VVP_loc_buf1, r2_buf1)
    knl_5C_1 = VVP_C_prg.VVP_C
    knl_5C_1.set_args(r2_buf1, r2_size1, VVP_loc_buf1, 
                      gamma1_dev1_buf, neg_gamma1_dev1_buf)
    knl_9A_1 = VVP_A_prg.VVP_A
    knl_9A_1.set_args(r_buf1, s_buf1, np.uint32(0), m1_inner, VVP_loc_buf1, r1_buf1)
    knl_9B_1 = VVP_reduce_prg.VVP_reduce
    knl_9B_1.set_args(r1_buf1, r1_size1, VVP_loc_buf1, r2_buf1)
    knl_9C_1 = VVP_reduce_prg.VVP_reduce
    knl_9C_1.set_args(r2_buf1, r2_size1, VVP_loc_buf1, delta1_new_dev1_buf) 
    
    knl_3A_2 = VVP_A_prg.VVP_A
    knl_3A_2.set_args(r_buf2, d_buf2, slice_size, m2_inner, VVP_loc_buf2, r1_buf2)
    knl_3B_2 = VVP_reduce_prg.VVP_reduce
    knl_3B_2.set_args(r1_buf2, r1_size2, VVP_loc_buf2, r2_buf2)
    knl_3C_2 = VVP_reduce_prg.VVP_reduce
    knl_3C_2.set_args(r2_buf2, r2_size2, VVP_loc_buf2, delta2_new_dev2_buf)
    knl_5A_2 = VVP_A_prg.VVP_A
    knl_5A_2.set_args(d_buf2, q_buf2, slice_size, m2_inner, VVP_loc_buf2, r1_buf2)
    knl_5B_2 = VVP_reduce_prg.VVP_reduce
    knl_5B_2.set_args(r1_buf2, r1_size2, VVP_loc_buf2, r2_buf2)
    knl_5C_2 = VVP_C_prg.VVP_C
    knl_5C_2.set_args(r2_buf2, r2_size2, VVP_loc_buf2, 
                      gamma2_dev2_buf, neg_gamma2_dev2_buf)
    knl_9A_2 = VVP_A_prg.VVP_A
    knl_9A_2.set_args(r_buf2, s_buf2, slice_size, m2_inner, VVP_loc_buf2, r1_buf2)
    knl_9B_2 = VVP_reduce_prg.VVP_reduce
    knl_9B_2.set_args(r1_buf2, r1_size2, VVP_loc_buf2, r2_buf2)
    knl_9C_2 = VVP_reduce_prg.VVP_reduce
    knl_9C_2.set_args(r2_buf2, r2_size2, VVP_loc_buf2, delta2_new_dev2_buf)
    

In [ ]:
# Conjugate gradient to solve A*x = b (or for us, A*u_i = L*u_{i-1} + N)

# Initialization
tol = 1e-6
i_max = 100
t = 0
total_iterations = 0 
        
# Reset u0 and x
u0_init_event_1 = cl.enqueue_copy(queue1, u0_buf1, np.zeros_like(u_0[0:m1]))
u0_init_event_2 = cl.enqueue_copy(queue1, u0_buf2, np.zeros_like(u_0[nVertices-m2:nVertices]), wait_for = [u0_init_event_1])
x_init_event_1 = cl.enqueue_copy(queue1, x_buf1, np.zeros_like(u_0[0:m1]), wait_for = [u0_init_event_2])
x_init_event_2 = cl.enqueue_copy(queue1, x_buf2, np.zeros_like(u_0[nVertices-m2:nVertices]), wait_for = [x_init_event_1])

# Apply corrosion profile for coefficient scaling of A and L, and gather data for preconditioner P
event_PA_1 = cl.enqueue_nd_range_kernel(queue1, knl_PA_1, (MVP_global_size1,), (MVP_wg_size,))
event_PA_2 = cl.enqueue_nd_range_kernel(queue2, knl_PA_2, (MVP_global_size2,), (MVP_wg_size,))
# Consolidate P
event_PB_1 = cl.enqueue_nd_range_kernel(queue1, knl_PB_1, (m1,), None, wait_for = [event_PA_1])
event_PB_2 = cl.enqueue_nd_range_kernel(queue2, knl_PB_2, (m2,), None, wait_for = [event_PA_2])

while(t <= 50*dt):
    #######################TRANSFER#######################################
    event_u01_load_buf = cl.enqueue_copy(queue1, u0_x_boundary_buf1, u0_buf1, byte_count = slice_size*4,
                      src_offset = (m1_inner-slice_size)*4, dest_offset = 0, wait_for = [])
    event_x1_load_buf = cl.enqueue_copy(queue1, u0_x_boundary_buf1, x_buf1, byte_count = slice_size*4,
                      src_offset = (m1_inner-slice_size)*4, dest_offset = slice_size*4, wait_for = [])
    event_u02_load_buf = cl.enqueue_copy(queue2, u0_x_boundary_buf2, u0_buf2, byte_count = slice_size*4,
                      src_offset = slice_size*4, dest_offset = 0, wait_for = [])
    event_x2_load_buf = cl.enqueue_copy(queue2, u0_x_boundary_buf2, x_buf2, byte_count = slice_size*4,
                      src_offset = slice_size*4, dest_offset = slice_size*4, wait_for = [])
        
    event_u01_swap = cl.enqueue_copy(queue2, u0_buf2, u0_x_boundary_buf1, byte_count = slice_size*4,
                              src_offset = 0, dest_offset = 0, wait_for = [event_u01_load_buf, event_x1_load_buf])
    event_x1_swap = cl.enqueue_copy(queue2, x_buf2, u0_x_boundary_buf1, byte_count = slice_size*4,
                              src_offset = slice_size*4, dest_offset = 0, wait_for = [event_u01_load_buf, event_x1_load_buf])
    event_u02_swap = cl.enqueue_copy(queue1, u0_buf1, u0_x_boundary_buf2, byte_count = slice_size*4,
                              src_offset = 0, dest_offset = m1_inner*4, wait_for = [event_u02_load_buf, event_x2_load_buf])
    event_x2_swap = cl.enqueue_copy(queue1, x_buf1, u0_x_boundary_buf2, byte_count = slice_size*4,
                              src_offset = slice_size*4, dest_offset = m1_inner*4, wait_for = [event_u02_load_buf, event_x2_load_buf])    
    
    # Compute RHS vector for PCG step i, b = L*u_{i-1} + N
    event_RHS_A_1 = cl.enqueue_nd_range_kernel(queue1, knl_RHS_A_1, (MVP_global_size1,), (MVP_wg_size,), wait_for = [event_u02_swap])
    event_RHS_A_2 = cl.enqueue_nd_range_kernel(queue2, knl_RHS_A_2, (MVP_global_size2,), (MVP_wg_size,), wait_for = [event_u01_swap])
    # Consolidate b
    event_RHS_B_1 = cl.enqueue_nd_range_kernel(queue1, knl_RHS_B_1, (m1,), None, wait_for = [event_RHS_A_1])
    event_RHS_B_2 = cl.enqueue_nd_range_kernel(queue2, knl_RHS_B_2, (m2,), None, wait_for = [event_RHS_A_2])

    # Preconditioned Conjugate Gradient to solve A*x = b
    tic_PCG = time.clock()
    i = 0

    # 1A) Ax = A*x_init      (MVP)
    event_1A_1 = cl.enqueue_nd_range_kernel(queue1, knl_1A_1, (MVP_global_size1,), (MVP_wg_size,), wait_for = [event_x2_swap])
    event_1A_2 = cl.enqueue_nd_range_kernel(queue2, knl_1A_2, (MVP_global_size2,), (MVP_wg_size,), wait_for = [event_x2_swap])

    # 1B) r = b - Ax      (VAVSP)
    event_1B_1 = cl.enqueue_nd_range_kernel(queue1, knl_1B_1, (m1,), None, wait_for = [event_1A_1, event_RHS_B_1])
    event_1B_2 = cl.enqueue_nd_range_kernel(queue2, knl_1B_2, (m2,), None, wait_for = [event_1A_2, event_RHS_B_2])
    
    # 2) d = Pinv * r      (DIMVP)
    event_2_1 = cl.enqueue_nd_range_kernel(queue1, knl_2_1, (m1,), None, wait_for = [event_1B_1, event_PB_1])
    event_2_2 = cl.enqueue_nd_range_kernel(queue2, knl_2_2, (m2,), None, wait_for = [event_1B_2, event_PB_2])

    # 3) delta_new = r' * d      (VVP)
    if (reduction_steps == 1):
        event_3A_1 = cl.enqueue_nd_range_kernel(queue1, knl_3A_1, (global_red_size1,), (max_wg_size,), wait_for = [event_2_1])
        event_3A_2 = cl.enqueue_nd_range_kernel(queue2, knl_3A_2, (global_red_size2,), (max_wg_size,), wait_for = [event_2_2])
        event_3C_1 = cl.enqueue_nd_range_kernel(queue1, knl_3B_1, (max_wg_size,), (max_wg_size,), wait_for = [event_3A_1])
        event_3C_2 = cl.enqueue_nd_range_kernel(queue2, knl_3B_2, (max_wg_size,), (max_wg_size,), wait_for = [event_3A_2])
    elif (reduction_steps == 2):
        event_3A_1 = cl.enqueue_nd_range_kernel(queue1, knl_3A_1, (global_red1_size1,), (max_wg_size,), wait_for = [event_2_1])
        event_3A_2 = cl.enqueue_nd_range_kernel(queue2, knl_3A_2, (global_red1_size2,), (max_wg_size,), wait_for = [event_2_2])
        event_3B_1 = cl.enqueue_nd_range_kernel(queue1, knl_3B_1, (global_red2_size1,), (max_wg_size,), wait_for = [event_3A_1])
        event_3B_2 = cl.enqueue_nd_range_kernel(queue2, knl_3B_2, (global_red2_size2,), (max_wg_size,), wait_for = [event_3A_2])
        event_3C_1 = cl.enqueue_nd_range_kernel(queue1, knl_3C_1, (max_wg_size,), (max_wg_size,), wait_for = [event_3B_1])
        event_3C_2 = cl.enqueue_nd_range_kernel(queue2, knl_3C_2, (max_wg_size,), (max_wg_size,), wait_for = [event_3B_2])
        
    # Map delta to host
    #######################TRANSFER#######################################
    event_mapDelta_1 = cl.enqueue_copy(queue1, delta1_new_dev1, delta1_new_dev1_buf, wait_for = [event_3C_1]) 
    event_mapDelta_2 = cl.enqueue_copy(queue1, delta2_new_dev2, delta2_new_dev2_buf, wait_for = [event_3C_2]) 
    
    event_delta1_swap = cl.enqueue_copy(queue1, delta1_new_dev2_buf, delta1_new_dev1_buf, wait_for = [event_3C_1])
    event_delta2_swap = cl.enqueue_copy(queue1, delta2_new_dev1_buf, delta2_new_dev2_buf, wait_for = [event_3C_2])
    queue1.flush()
    
    # Inner loop:
    while (i < i_max) & (np.sqrt(delta1_new_dev1 + delta2_new_dev2) > tol):
        # 4) q = A * d      (MVP)
        #######################TRANSFER#######################################
        event_d1_load_buf = cl.enqueue_copy(queue1, d_boundary_buf1, d_buf1, byte_count = slice_size*4,
                              src_offset = (m1_inner-slice_size)*4, dest_offset = 0, wait_for = [])
        event_d2_load_buf = cl.enqueue_copy(queue2, d_boundary_buf2, d_buf2, byte_count = slice_size*4,
                              src_offset = slice_size*4, dest_offset = 0, wait_for = [])
        event_d1_swap = cl.enqueue_copy(queue2, d_buf2, d_boundary_buf1, byte_count = slice_size*4,
                              src_offset = 0, dest_offset = 0, wait_for = [event_d1_load_buf])
        event_d2_swap = cl.enqueue_copy(queue1, d_buf1, d_boundary_buf2, byte_count = slice_size*4,
                              src_offset = 0, dest_offset = m1_inner*4, wait_for = [event_d2_load_buf])
        
        event_4A_1 = cl.enqueue_nd_range_kernel(queue1, knl_4A_1, (MVP_global_size1,), (MVP_wg_size,), wait_for = [event_d2_swap ])
        event_4A_2 = cl.enqueue_nd_range_kernel(queue2, knl_4A_2, (MVP_global_size2,), (MVP_wg_size,), wait_for = [event_d1_swap ])

        event_4B_1 = cl.enqueue_nd_range_kernel(queue1, knl_4B_1, (m1,), None, wait_for = [event_4A_1])
        event_4B_2 = cl.enqueue_nd_range_kernel(queue2, knl_4B_2, (m2,), None, wait_for = [event_4A_2])

        # delta_old = delta_new  
        #######################Within-device-copy#######################################
        event_delta1_update1 = cl.enqueue_copy(queue1, delta1_dev1_buf, delta1_new_dev1_buf, wait_for = [])
        event_delta2_update1 = cl.enqueue_copy(queue1, delta2_dev1_buf, delta2_new_dev1_buf, wait_for = [])
        event_delta1_update2 = cl.enqueue_copy(queue2, delta1_dev2_buf, delta1_new_dev2_buf, wait_for = [])
        event_delta2_update2 = cl.enqueue_copy(queue2, delta2_dev2_buf, delta2_new_dev2_buf, wait_for = [])
        
        # 5) gamma = (d' * q)      (VVP)
        if (reduction_steps == 1):
            event_5A_1 = cl.enqueue_nd_range_kernel(queue1, knl_5A_1, (global_red_size1,), (max_wg_size,), wait_for = [event_4B_1])
            event_5A_2 = cl.enqueue_nd_range_kernel(queue2, knl_5A_2, (global_red_size2,), (max_wg_size,), wait_for = [event_4B_2])
            event_5C_1 = cl.enqueue_nd_range_kernel(queue1, knl_5B_1, (max_wg_size,), (max_wg_size,), wait_for = [event_5A_1])    
            event_5C_2 = cl.enqueue_nd_range_kernel(queue2, knl_5B_2, (max_wg_size,), (max_wg_size,), wait_for = [event_5A_2])
        elif (reduction_steps == 2):
            event_5A_1 = cl.enqueue_nd_range_kernel(queue1, knl_5A_1, (global_red1_size1,), (max_wg_size,), wait_for = [event_4B_1])
            event_5A_2 = cl.enqueue_nd_range_kernel(queue2, knl_5A_2, (global_red1_size2,), (max_wg_size,), wait_for = [event_4B_2])
            event_5B_1 = cl.enqueue_nd_range_kernel(queue1, knl_5B_1, (global_red2_size1,), (max_wg_size,), wait_for = [event_5A_1]) 
            event_5B_2 = cl.enqueue_nd_range_kernel(queue2, knl_5B_2, (global_red2_size2,), (max_wg_size,), wait_for = [event_5A_2])    
            event_5C_1 = cl.enqueue_nd_range_kernel(queue1, knl_5C_1, (max_wg_size,), (max_wg_size,), wait_for = [event_5B_1])    
            event_5C_2 = cl.enqueue_nd_range_kernel(queue2, knl_5C_2, (max_wg_size,), (max_wg_size,), wait_for = [event_5B_2])

        #######################TRANSFER#######################################
        event_gamma1_swap = cl.enqueue_copy(queue1, gamma1_dev2_buf, gamma1_dev1_buf, wait_for = [event_5C_1])
        event_neg_gamma1_swap = cl.enqueue_copy(queue1, neg_gamma1_dev2_buf, neg_gamma1_dev1_buf, wait_for = [event_5C_1])
        event_gamma2_swap = cl.enqueue_copy(queue1, gamma2_dev1_buf, gamma2_dev2_buf, wait_for = [event_5C_2])
        event_neg_gamma2_swap = cl.enqueue_copy(queue1, neg_gamma2_dev1_buf, neg_gamma2_dev2_buf, wait_for = [event_5C_2])
        event_gamma1_swap.wait()
        event_neg_gamma1_swap.wait()
        event_gamma2_swap.wait()
        event_neg_gamma2_swap.wait()
        
        # 6) x = x + delta_new/gamma * d      (VAVSP)
        event_6_1 = cl.enqueue_nd_range_kernel(queue1, knl_6_1, (m1,), None, wait_for = [])
        event_6_2 = cl.enqueue_nd_range_kernel(queue2, knl_6_2, (m2,), None, wait_for = [])

        
        # Optional correction step
        if ((i % 200) == 1):
        # 7) r = b - A*x      (MVP, VAVSP)
            event_d1_swap = cl.enqueue_copy(queue2, d_buf2, d_buf1, byte_count = slice_size*4,
                                  src_offset = m*4, dest_offset = 0, wait_for = [event_6_1])
            event_d2_swap = cl.enqueue_copy(queue1, d_buf1, d_buf2, byte_count = slice_size*4,
                                  src_offset = 0, dest_offset = m*4, wait_for = [event_6_1])
            event_d1_swap.wait()
            event_d2_swap.wait()
            event_7A_1 = cl.enqueue_nd_range_kernel(queue1, knl_7A_1, (MVP_global_size1,), (MVP_wg_size,), wait_for = [event_d2_swap])
            event_7A_2 = cl.enqueue_nd_range_kernel(queue2, knl_7A_2, (MVP_global_size2,), (MVP_wg_size,), wait_for = [event_d1_swap])
            event_7B_1 = cl.enqueue_nd_range_kernel(queue1, knl_7B_1, (m1,), None, wait_for = [event_7A_1])
            event_7B_2 = cl.enqueue_nd_range_kernel(queue2, knl_7B_2, (m2,), None, wait_for = [event_7A_2])
        else:
        # 7) r = r - alpha*q      (VAVSP)
            event_7B_1 = cl.enqueue_nd_range_kernel(queue1, knl_7_1, (m1,), None, wait_for = [event_6_1])
            event_7B_2 = cl.enqueue_nd_range_kernel(queue2, knl_7_2, (m2,), None, wait_for = [event_6_2])
            

        
        # 8) s = Pinv * r      (DIMVP)
        event_8_1 = cl.enqueue_nd_range_kernel(queue1, knl_8_1, (m1,), None, wait_for = [event_7B_1])
        event_8_2 = cl.enqueue_nd_range_kernel(queue2, knl_8_2, (m2,), None, wait_for = [event_7B_2])
        
        # 9) delta_new = r' * s       (VVP)
        if (reduction_steps == 1):
            event_9A_1 = cl.enqueue_nd_range_kernel(queue1, knl_9A_1, (global_red_size1,), (max_wg_size,), wait_for = [event_8_1])
            event_9A_2 = cl.enqueue_nd_range_kernel(queue2, knl_9A_2, (global_red_size2,), (max_wg_size,), wait_for = [event_8_2])
            event_9C_1 = cl.enqueue_nd_range_kernel(queue1, knl_9B_1, (max_wg_size,), (max_wg_size,), wait_for = [event_9A_1])
            event_9C_2 = cl.enqueue_nd_range_kernel(queue2, knl_9B_2, (max_wg_size,), (max_wg_size,), wait_for = [event_9A_2])
        elif (reduction_steps == 2):
            event_9A_1 = cl.enqueue_nd_range_kernel(queue1, knl_9A_1, (global_red1_size1,), (max_wg_size,), wait_for = [event_8_1])
            event_9A_2 = cl.enqueue_nd_range_kernel(queue2, knl_9A_2, (global_red1_size2,), (max_wg_size,), wait_for = [event_8_2])
            event_9B_1 = cl.enqueue_nd_range_kernel(queue1, knl_9B_1, (global_red2_size1,), (max_wg_size,), wait_for = [event_9A_1])
            event_9B_2 = cl.enqueue_nd_range_kernel(queue2, knl_9B_2, (global_red2_size2,), (max_wg_size,), wait_for = [event_9A_2])
            event_9C_1 = cl.enqueue_nd_range_kernel(queue1, knl_9C_1, (max_wg_size,), (max_wg_size,), wait_for = [event_9B_1])
            event_9C_2 = cl.enqueue_nd_range_kernel(queue2, knl_9C_2, (max_wg_size,), (max_wg_size,), wait_for = [event_9B_2])


        # 10) beta = delta_new/delta_old
        #######################TRANSFER#######################################
        event_delta1_swap = cl.enqueue_copy(queue1, delta1_new_dev2_buf, delta1_new_dev1_buf, wait_for = [event_9C_1])
        event_delta2_swap = cl.enqueue_copy(queue1, delta2_new_dev1_buf, delta2_new_dev2_buf, wait_for = [event_9C_2])

        
        # 11) d = s + beta * d      (VAVSP)
        event_11_1 = cl.enqueue_nd_range_kernel(queue1, knl_11_1, (m1,), None, wait_for = [event_delta2_swap])
        event_11_2 = cl.enqueue_nd_range_kernel(queue2, knl_11_2, (m2,), None, wait_for = [event_delta1_swap])

        
        # Map delta to host, sometimes
        if ((i % 1) == 0):
            #######################TRANSFER#######################################
            event_mapDelta_1 = cl.enqueue_copy(queue1, delta1_new_dev1, delta1_new_dev1_buf, wait_for = [event_11_1])
            event_mapDelta_2 = cl.enqueue_copy(queue2, delta2_new_dev2, delta2_new_dev2_buf, wait_for = [event_11_2])

        i += 1
        
        # Store inner loop profiling info
        event_11_1.wait()
        event_11_2.wait()
        
        #################DELETE EVENTS#############################
        del event_d1_swap, event_d2_swap, event_d1_load_buf, event_d2_load_buf
        del event_gamma2_swap, event_neg_gamma2_swap, event_gamma1_swap, event_neg_gamma1_swap
        del event_delta1_update1, event_delta2_update1, event_delta1_update2, event_delta2_update2 

    # Set u_{i-1} = u_i
    event_u0_1 = cl.enqueue_nd_range_kernel(queue1, knl_u0_1, (m1,), None)
    event_u0_2 = cl.enqueue_nd_range_kernel(queue2, knl_u0_2, (m2,), None)

    total_iterations += i
    t += dt
    
    event_u0_1.wait()
    event_u0_2.wait()
        
print("Total iterations: %g" % (total_iterations))